This project is inspired by the CS50x Fiftyville Problem Set;
https://cs50.harvard.edu/x/2022/psets/7/fiftyville/

```bash
Libraries: Install
```

In [1]:
!pip install --upgrade --quiet langchain langchain-community langchain-openai langchainhub

```bash
Libraries: Import
```

In [2]:
import getpass
import os
from langchain.chains import create_sql_query_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_community.utilities import SQLDatabase
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from operator import itemgetter

```bash
Connect: OpenAI
```

In [3]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

# Choose model
llm = ChatOpenAI(model="gpt-3.5-turbo")

```bash
Connect: Database
```

In [4]:
db = SQLDatabase.from_uri("sqlite:///data/fiftyville.db")

```js
Inspect Database
```

In [5]:
print(db.dialect)

sqlite


In [6]:
print(db.get_usable_table_names())

['airports', 'atm_transactions', 'bakery_security_logs', 'bank_accounts', 'crime_scene_reports', 'flights', 'interviews', 'passengers', 'people', 'phone_calls']


In [7]:
print("\nColumn names: airport")
db.run("SELECT name FROM PRAGMA_TABLE_INFO('crime_scene_reports');")


Column names: airport


"[('id',), ('year',), ('month',), ('day',), ('street',), ('description',)]"

In [8]:
db.run("SELECT * from crime_scene_reports LIMIT 5;")

"[(1, 2021, 1, 1, 'Chamberlin Street', 'Credit card fraud took place at 00:53. Two people witnessed the event.'), (2, 2021, 1, 1, 'Zlatkova Street', 'Burglary took place at 05:23. Two people witnessed the event.'), (3, 2021, 1, 1, 'Bowden Boulevard', 'Shoplifting took place at 20:48. Two people witnessed the event.'), (4, 2021, 1, 1, 'Boyce Avenue', 'Jaywalking took place at 23:37. Two people witnessed the event.'), (5, 2021, 1, 2, 'Widenius Street', 'Jaywalking took place at 20:33. One person witnessed the incident.')]"

```js
Chain: Generate SQL Query
```

In [9]:
# 'create_sql_query_chain' to generate SQL Query for a database from natural language
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many people came into the parking lot?"})
response

"SELECT COUNT(DISTINCT license_plate) as total_people\nFROM bakery_security_logs\nWHERE activity = 'entrance';"

In [10]:
# test the generated query on the database
db.run(response)

'[(140,)]'

```js
Chain: query generation and query execusion 
```

In [11]:
# tool to execute a given query on a database
execute_query = QuerySQLDataBaseTool(db=db)
# and the query generator
write_query = create_sql_query_chain(llm, db)
# chain both together
chain = write_query | execute_query

In [12]:
chain.invoke({"question": "How many flights are there?"})

'[(58,)]'

```js
Next Steps:- 
Chain: generate and execute query with custom prompt
- Prompt
```

In [13]:
detective_prompt = PromptTemplate.from_template(
    """
You are a detective tasked with solving the mystery of the stolen CS50 Duck in Fiftyville. Use the provided database schema to write and execute SQL queries that will help you find the answers to the following questions:

1. Who the thief is,
2. What city the thief escaped to, and
3. Who the thief’s accomplice is who helped them escape.

The theft took place on July 28, 2021, on Humphrey Street. The thief stole the duck and took a flight out of town shortly afterward with the help of an accomplice.

Here are the relevant tables in the Fiftyville database:
- crime_scene_reports(id INTEGER, year INTEGER, month INTEGER, day INTEGER, street TEXT, description TEXT)
- interviews(id INTEGER, name TEXT, year INTEGER, month INTEGER, day INTEGER, transcript TEXT)
- atm_transactions(id INTEGER, account_number INTEGER, year INTEGER, month INTEGER, day INTEGER, atm_location TEXT, transaction_type TEXT, amount INTEGER)
- bank_accounts(account_number INTEGER, person_id INTEGER, creation_year INTEGER)
- airports(id INTEGER, abbreviation TEXT, full_name TEXT, city TEXT)
- flights(id INTEGER, origin_airport_id INTEGER, destination_airport_id INTEGER, year INTEGER, month INTEGER, day INTEGER, hour INTEGER, minute INTEGER)
- passengers(flight_id INTEGER, passport_number INTEGER, seat TEXT)
- phone_calls(id INTEGER, caller TEXT, receiver TEXT, year INTEGER, month INTEGER, day INTEGER, duration INTEGER)
- people(id INTEGER, name TEXT, phone_number TEXT, passport_number INTEGER, license_plate TEXT)
- bakery_security_logs(id INTEGER, year INTEGER, month INTEGER, day INTEGER, hour INTEGER, minute INTEGER, activity TEXT, license_plate TEXT)

Use this context to answer the following user questions by creating the necessary SQL queries and interpreting the SQL results to guide the investigation.

Question: {question}

SQL Query: {query}
SQL Result: {result}
Answer: 
"""
)

```js
- Chain
```

In [14]:
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | detective_prompt
    | llm
    | StrOutputParser()
)

```js
- Invoke
```

In [15]:
chain.invoke({"question": "What time was the crime reported and commited?"})

'The crime of the stolen CS50 Duck was reported on July 28, 2021. The exact time of the crime was not provided in the SQL query results, so further investigation or querying may be needed to determine the specific time the crime was committed.'

In [16]:
chain.invoke({"question": "What else happended around the time the crime was committed?"})

'Around the time the crime was committed on July 28, 2021, other crimes took place in Fiftyville. These included credit card fraud on Chamberlin Street at 00:53, burglary on Zlatkova Street at 05:23, shoplifting on Bowden Boulevard at 20:48, and jaywalking on Boyce Avenue at 23:37. Each of these incidents had two witnesses present.'